# CoursewareHub-実行状況の可視化

---
CoursewareHubのsingle-userサーバからnotebookの実行ログを収集し、各ユーザの実行状況を可視化します。

## 概要

Jupyter Notebook環境に[Jupyter-LC_wrapper](https://github.com/NII-cloud-operation/Jupyter-LC_wrapper)（以下でlc_wrapperと記す）を追加するとnotebookの実行結果に関する情報をログとして記録できるようになります。このログには、実行したセルを特定するMEME ID、実行したユーザのUID、実行したコード、実行開始時刻、実行終了時刻などの情報が記録されます。CoursewareHubが実行するsingle-userサーバからlc_wrapperのログを収集することにより各ユーザのnotebook実行状況を把握することができます。

このnotebookではsingle-userサーバのlc_wrapperのログをfluentdに送信し、そのログをElasticsearchで収集する環境を構築します。また収集したログをKibanaで可視化し、各ユーザの進捗状況を確認できるようにします。

![全体構成](images/fluentd-201-01.png)

### 前提条件

このnotebookを実行するための前提条件を以下に示します。

* ログの送信元となるCoursewareHubが構築済みであること
* ログの収集先となるElasticsearch、可視化で利用するKibanaが構築済みであること
* CoursewareHubのsingle-userサーバでlc_wrapperが有効になっていること


Elasticsearch, Kibanaの構築には以下に示すnotebookを利用することができます。

* [801-開発用Elasticsearch,Kibanaの構築.ipynb](notebook/801-開発用Elasticsearch%2CKibanaの構築.ipynb)

ただし、上記のnotebookで構築するElasticsearch, Kibanaは開発用の簡易構成となっているため実運用には適していません。

### lc_wrapper

single-userサーバからnotebookの実行ログを取得するためにlc_wrapperを利用します。

CoursewareHubが実行するデフォルトのsingle-userサーバではlc_wrapperが有効になっています。

CoursewareHubのテンプレートにあるnotebook「[731-講義用Notebook環境のイメージ登録.ipynb](https://github.com/nii-gakunin-cloud/ocs-templates/blob/master/CoursewareHub/notebooks/731-%E8%AC%9B%E7%BE%A9%E7%94%A8Notebook%E7%92%B0%E5%A2%83%E3%81%AE%E3%82%A4%E3%83%A1%E3%83%BC%E3%82%B8%E7%99%BB%E9%8C%B2.ipynb)」の手順でsingle-userサーバのコンテナイメージをカスタマイズする場合は、以下のモジュールを追加するようにしてください。

* [Jupyter-LC_wrapper](https://github.com/NII-cloud-operation/Jupyter-LC_wrapper)
* [Jupyter-LC_nblineage](https://github.com/NII-cloud-operation/Jupyter-LC_nblineage)
* [Jupyter-multi_outputs](https://github.com/NII-cloud-operation/Jupyter-multi_outputs)

カスタムイメージの定義例として、以下に示す２つのレポジトリがあります。

* [cwh-custom-image-seaborn](https://github.com/nii-gakunin-cloud/cwh-custom-image-seaborn.git)
* [cwh-custom-image-network-command](https://github.com/nii-gakunin-cloud/cwh-custom-image-network-command.git)

`main`ブランチでは簡潔な例を示すためlc_wrapperが追加されていません。それぞれのレポジトリには`lc_wrapper`ブランチがあり`main`ブランチの内容にlc_wrapperを追加されたものとなっています。

`lc_wrapper`ブランチの定義をカスタムイメージとして登録するにはCoursewareHubの管理画面の[Environments]から[Add New]ボタンを選択すると表示される[Create Environment]ダイアログで**Reference (git commit)**欄にブランチ名を入力してください。

![CoursewareHubイメージビルド](images/fluentd-201-02.png)

ビルドしたコンテナイメージでlc_wrapperが有効になっていることを確認するには、起動したsingle-userサーバのterminalで`jupyter-nbextension list`を実行してください。以下のような表示となり`lc_wrapper`が有効になっていることが確認できます。

```console
$ jupyter-nbextension list
Known nbextensions:
  config dir: /srv/conda/envs/notebook/etc/jupyter/nbconfig
    notebook section
      jupyter_resource_usage/main  enabled
      - Validating: OK
      jupyter-offlinenotebook/main  enabled
      - Validating: OK
      jupyter-js-widgets/extension  enabled
      - Validating: OK
      nblineage/main  enabled
      - Validating: OK
      multi_outputs/main  enabled
      - Validating: OK
      lc_wrapper/main  enabled
      - Validating: OK
```

### ログデータ

Elasticsearchで収集するログデータにどのような値が含まれているのかについて説明します。

#### lc_wrapper

single-userサーバでnotebookのコードセルを実行するとlc_wrapperにより実行ログがfluentdに送信されます。送信されるログに含まれている主な値を次表に示します。

|フィールド名|説明|例|
|:---|:---|:---|
|lc_cell_meme|セルのMEME ID|1a339f04-3e8f-11ef-91c2-02420a640009-3-c957-e4b7-54b0|
|start_time|セルの実行開始時刻|Sep 1, 2024 @ 13:30:00.000|
|end_time|セルの実行終了時刻|Sep 1, 2024 @ 13:30:00.000|
|execute_reply_status|実行状態|ok|
|input|セルの入力値|print(math.sin(1)); math.cos(1)|
|output|コードの実行による標準出力、標準エラー|0.8414709848078965|
|result|セルの出力結果を記録したパス|/home/user01xacca0c/python-basic/.log/20240901/20240901-043000-0280-0.pkl|
|uid|利用者のUID|1001|
|gid|利用者のGID|1100|
|lc_notebook_meme|notebookのMEME ID|1a339e00-3e8f-11ef-91c2-02420a640009|
|notebook_path|notebookのパス|ex01.ipynb|
|server_signature|サーバのsignature|89e27d08-7a39-11ef-a663-02420a640015|

lc_wrapperが記録する`start_time`, `end_time`の値はエポック秒です。Elasticsearchではそれらの値を`date`型としてパースするので日時値として表示されます。

`output`にはセルを実行した際に標準出力、標準エラーに出力された値が記録されます。セルの実行結果として表示される`Out[1]`などの値はlc_wrapperがfluentdに送信するログからは除外されています。それらの値はfluentdに送信されるログとは別にsingle-userサーバのファイルに記録されます。fluentdに送信されるログにはセルの実行結果を記録したファイルのパスのみが`result`フィールドに記録されます（セルの出力結果がある場合）。


#### セルの実行結果

lc_wrapperにより別ファイルに記録されたセルの実行結果の内容を読み込みfluentdの送信ログに追加します。この追加処理はfluentdのfilterプラグインで実施します。

追加するフィールドを次表に示します。lc_wrapperログの`result`フィールドに対応する値を追加します。

|フィールド名|説明|例|
|:---|:---|:---|
|result_json|セル出力結果をJSONエンコードした値|{"content":{"execution_count":13,"metadata":{},"data":{"text/plain":"0.5403023058681398"}},"msg_type":"execute_result"}|
|result_text|セルの出力結果テキスト部分|0.5403023058681398|

セルの出力結果はlc_wrapperによって[pickle](https://docs.python.org/ja/3.13/library/pickle.html)形式で直列化されファイルに保存されています。このファイルのパスはfluentdのログの`result`フィールドに記録されています。fluentdのfilterプラグインでこのファイルを読み込み、内容をJSON形式に変換したものを`result_json`フィールドに追加します。さらに`result_json`からテキスト部分（"text/plain"）のみを抽出し`result_text`フィールドとして追加します。

`result_text`として`result_json`からどの部分を抽出するかについては「[5.2 セル実行結果からテキスト部分の抽出](#セル実行結果からテキスト部分の抽出)」で設定することができます。

#### ユーザ情報

lc_wrapperから送信されるログにはユーザ情報として`uid`フィールドしか含まれていません。ユーザを識別しやすくするため、CoursewareHubのmanagerノードのpasswdを参照し、アカウント名などの情報を付加します。この追加処理はfluentdのfilterプラグインで実施します。

追加するフィールドを次表に示します。lc_wrapperログの`uid`フィールドに対応する値を追加します。

|フィールド名|説明|例|
|:---|:---|:---|
|user_name|アカウント名|user01xacca0c|
|local_name|メールアドレスのローカルパート|user01|

CoursewareHubではユーザの識別子としてメールアドレスを用いています。上記の表の`local_name`は、このメールアドレスからローカルパートを抜き出した値となっています。そのため`local_name`の値は複数のユーザで重複する可能性があることに注意してください。

#### セルの通番

lc_wrapperログの`lc_cell_meme`フィールドの値を確認することで、single-userサーバでどのセルが実行されたのかを特定することができます。しかし、各ユーザの進捗状況を可視化するには単にセルを特定するだけではなく、notebookにおけるセルの記述順序などの情報もあった方が処理しやすくなります。

そこでCoursewareHubの教材としてリリースされたnotebookが配置されているディレクトリ`/jupyter/admin/textbook`からコードセルのMEME IDを収集し、それらのセルに通番を割り当てる処理を事前に行います。収集した情報はCSVファイルとして保存しておき、fluentdのfilterプラグインでlc_wrapperログの`lc_cell_meme`に対応するフィールドを追加するようにします。

追加するフィールドを次表に示します。

|フィールド名|説明|例|
|:---|:---|:---|
|qno|コードセルの通し番号|00-000|
|qlabel|実行したセルの直前にあるmarkdownセルから抽出した文字列|問1-1|
|lc_cell_meme_uuid|セルのMEME IDからブランチ番号を除いた値|1a339f04-3e8f-11ef-91c2-02420a640009|
|textbook_path|リリースしたtextbookのパス|python-basic/ex01.ipynb|
|course_server|CoursewareHubのコース名|python-basic|

`qno`はnotebook毎のコードセル順を得るための通し番号となっています。`-`の前にある値がnotebook毎に割り当てた番号で、後にある値がnotebook内のコードセルの通し番号となっています。

`qlabel`はコードセルの直前にあるmarkdownセルからパターンマッチにより抽出した文字列になります。これはmarkdownセルの記述内容から設問番号などの文字列を取得することを想定しています。このフィールドは「[3.1 MEME IDに関するパラメータの指定](#MEME-IDに関するパラメータの指定)」で`qlabel_pattern`を指定した場合にのみ追加されます。

`textbook_path`は、リリースした教材のディレクトリにあるnotebookのMEME IDとlc_wrapperログに記録された`lc_notebook_meme`が一致するファイルのパスになります。基本的には`notebook_path`と同じ値になりますが、[CoursewareHub](https://github.com/NII-cloud-operation/CoursewareHub-LC_platform)の複数コース機能を利用して運用している場合は`textbook_path`と`notebook_path`で異なる値が記録されます。`textbook_path`ではコース名に対応するディレクトリが付加された値になりますが、`notebook_path`はコース名部分が省かれたパスになります。

`course_server`は、`textbook_path`のディレクトリ部分を抽出した値になります。`textbook_path`にディレクトリ部分がないパスが記録されている場合は`course_server`の値は空になります。CoursewareHubの複数コース機能を利用して運用している場合、`course_server`にはコース名に相当する値が記録されることになります。

## 準備

構築対象となるホストやサービスのアドレスを指定してアクセスできることを確認します。

### パラメータの指定

接続先となるホストやサービスのアドレスなどを指定します。

#### CoursewareHub

CoursewareHubのmanagerノードに接続するためのパラメータを指定します。

managerノードのIPアドレスを次のセルで指定してください。

> 外部公開用のアドレスではなく内部ネットワークのIPアドレスを指定してください。

In [ ]:
# (例)
# cwh_hostname = "172.30.2.100"

cwh_hostname = 

接続先のユーザ名を指定してください。

In [ ]:
# (例)
# cwh_ssh_user = "vcp"

cwh_ssh_user = 

SSH公開鍵認証で接続するための秘密鍵のパスを指定してください。指定する値はこのnotebookを実行しているJupyter Notebook環境におけるパスを指定して下さい。また、ペアとなる公開鍵を接続先となるホストに事前に登録しておいてください。

In [ ]:
# (例)
# cwh_ssh_private_key_path = "~/.ssh/id_rsa"

cwh_ssh_private_key_path = 

接続先のCoursewareHubを表すansibleのグループ名を指定してください。このnotebookの以降の操作では、IPアドレスなどの値を直接指定せずに次のセルで指定されたグループ名を用いて操作を行います。

In [ ]:
# (例)
# cwh_group = "cwh"

cwh_group = 

#### Elasticsearch

ログの収集先となるElasticsearchのアドレスを指定します。

ElasticsearchのURLを指定してください。

In [ ]:
# (例)
# es_url = "http://172.30.2.101:9200"
# es_url = "https://172.30.2.101:9200"
# es_url = "https://es.example.org:9200"

es_url = 

以下の３つの項目（`es_ssl_verify`, `es_user`, `es_password`）は必要な場合のみ設定してください。「801-開発用Elasticsearch,Kibanaの構築.ipynb」を利用してElasticsearchを構築した場合は設定する必要はありません。

ssl verificationを無効にする場合は、次のセルのコメントを外して`es_ssl_verify`を`False`に設定してください。

In [ ]:
# es_ssl_verify = False

Elasticsearchのユーザ名を指定してください。

In [ ]:
# (例)
# es_user = "elastic"

es_user = 

Elasticsearchのパスワードを指定してください。

In [ ]:
from getpass import getpass

es_password = getpass()

#### Kibana

KibanaのURLを指定してください。

In [ ]:
# (例)
# kibana_url = "http://172.30.2.101:5601"
# kibana_url = "https://kibana.example.org"

kibana_url = 

### CoursewareHub

CoursewareHubのmanagerノードにansibleで接続する準備を行います。また接続したノードでCoursewareHubが実行されていることを確認します。

#### 設定ファイルの作成

ansibleの設定ファイルを作成します。

Ansibleのインベントリファイルを現在の作業ディレクトリに`inventory.yml`という名前で作成します。

In [ ]:
from pathlib import Path
import yaml

path_inventory = Path("inventory.yml")
if path_inventory.exists():
    with path_inventory.open() as f:
        inventory = yaml.safe_load(f)
else:
    inventory = {"all": {"children": {}}}
cwh = {
    cwh_group: {
        "hosts": {
            cwh_hostname: {
                "ansible_user": cwh_ssh_user,
                "ansible_ssh_private_key_file": str(Path(cwh_ssh_private_key_path).expanduser()),
            }
        }
    }
}
inventory["all"]["children"] |= cwh

with path_inventory.open(mode="w") as f:
    yaml.safe_dump(inventory, f, default_flow_style=False)

!cat inventory.yml

Ansibleの設定ファイル`ansible.cfg`にインベントリファイルのパスを設定します。

In [ ]:
import configparser

config = config = configparser.ConfigParser()
path_cfg = Path("ansible.cfg")
if path_cfg.exists():
    with path_cfg.open() as f:
        config.read_file(f)

if not config.has_section("defaults"):
    config.add_section("defaults")
config["defaults"]["inventory"] = str(path_inventory.resolve())

with path_cfg.open(mode="w") as f:
    config.write(f)

!cat ansible.cfg

`~/.ssh/known_hosts`に接続先となるホストのSSH鍵を登録します。

In [ ]:
!mkdir -p -m 0700 ~/.ssh
!touch ~/.ssh/known_hosts
!ssh-keygen -R {cwh_hostname}
!ssh-keyscan -H {cwh_hostname} >> ~/.ssh/known_hosts

#### 接続確認

対象となるホストへの接続確認を行います。

In [ ]:
!ansible {cwh_group} -m ping

管理者権限でコマンドを実行できることを確認します。

In [ ]:
!ansible {cwh_group} -b -a 'whoami'

#### 前提条件

接続先のホストでCoursewareHubが実行されていることを確認します。

次のセルを実行してエラーにならないことを確認してください。

In [ ]:
import json

!ansible {cwh_group} -a 'docker stack services coursewarehub'
out = !ansible {cwh_group} -a 'docker stack services --format json coursewarehub'
count = 0
for x in out[1:]:
    service = json.loads(x)
    if service["Name"].startswith("coursewarehub_") and service["Replicas"] == "1/1":
        count += 1
if count != 4:
    raise "CoursewareHubが実行されていません"

fluentdがsystemdのサービスとして実行されていることを確認します。

In [ ]:
!ansible {cwh_group} -b -a 'systemctl status fluentd'

### Elasticsearch, Kibana

Elasticsearch, Kibanaに接続できることを確認します。

接続ユーザ名などに応じたcurlのオプション文字列を作成します。

In [ ]:
es_opts = f'-u {es_user}:{es_password}' if 'es_password' in vars() else ''
if 'es_ssl_verify' in vars() and not es_ssl_verify:
    es_opts += ' -k'

Elasticsearchに接続できることを確認します。

In [ ]:
!curl {es_opts} {es_url}/_cluster/health?pretty

Kibanaに接続できることを確認します。

In [ ]:
!set -o pipefail; curl -sf {es_opts} {kibana_url}/api/status | jq "[.status.overall,.version]"

### パラメータの保存

ここまでに指定したパラメータの値をファイルに保存します。

CoursewareHubに接続するためのパラメータはansibleのインベントリファイル`inventory.yml`に記録されています。Elasticsearch, Kibanaに関するパラメータをgroup_varsとしてファイルに保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    cwh_group,
    _file='10-es.yml',
    es_url=es_url,
    kibana_url=kibana_url,
)
if 'es_password' in vars():
    update_group_vars(
        cwh_group,
        _file='10-es.yml',
        es_user=es_user,
        es_password=es_password,
    )
if 'es_ssl_verify' in vars():
    update_group_vars(
        cwh_group,
        _file='10-es.yml',
        es_ssl_verify=es_ssl_verify,
    )

## MEME IDの収集

リリースした教材からMEME IDを収集して通し番号を割り当てるサービスを配備します。

### MEME IDに関するパラメータの指定

MEME IDの収集は教材としてリリースしたnotebookが格納されているディレクトリ`/jupyter/admin/textbook`にある`*.ipynb`ファイルを対象としています。他のディレクトリでMEME IDを収集する場合は対象となるディレクトリを次のセルで指定してください。

In [ ]:
# (例)
# notebook_dir = "/jupyter/admin/textbook"
#
# notebook_dir = 

収集したMEME IDの値を保存するCSVファイルのパスを指定します。通常はデフォルトの格納場所`/var/lib/lc-wrapper/meme.csv`に保存されますが、他の場所にCSVファイルを保存する場合は次のセルでパスを指定してください。

In [ ]:
# (例)
# csv_file = "/var/lib/lc-wrapper/meme.csv"
#
# csv_file =

収集したログを可視化する際に、設問番号の情報などをmarkdownのセルから取得したいことがあります。次のセルに正規表現を指定すると、マッチする文字列をmarkdownセルから抽出し、CSVファイルに追加情報として保存されます。対象となるセルは実行したコードセルの前にあるmarkdownセルになります。また指定する正規表現は[Python](https://docs.python.org/ja/3/library/re.html)の指定方法に従ったものとしてください。ここで指定した正規表現により抽出した文字列は`qlabel`フィールドとしてElasticsearchに送信されます。

In [ ]:
# (例)
# qlabel_pattern = "問\s*\d+-\d+"
#
# qlabel_pattern = 

### パラメータの保存

指定されたパラメータを保存します。

In [ ]:
%run scripts/group.py

for name in ['notebook_dir', 'csv_file', 'qlabel_pattern']:
    if name in vars():
        params = {name: vars()[name]}
        update_group_vars(cwh_group, _file='10-meme.yml', **params)
    else:
        remove_group_vars(cwh_group, name)

!touch group_vars/{cwh_group}/10-meme.yml
!cat group_vars/{cwh_group}/10-meme.yml

### MEME IDの収集を実行する


MEME IDを収集しCSVファイルに保存するサービスを配備します。

配備するサービスではリリースした教材を配置するディレクトリ`/jupyter/admin/textbook`を監視します。ディレクトリ内にあるnotebookファイルが追加、変更されると、収集結果を保存するCSVファイルが更新されます。

In [ ]:
!ansible-playbook -l {cwh_group} playbooks/deploy-meme-csv.yml

配備したMEME ID収集コマンドを利用して対象となる教材に問題がないかチェックします。以下のチェックを行います。

* notebookにMEME IDが記録されているか
* 複数のnotebookに同じMEME IDが記録されていないか


In [ ]:
!ansible {cwh_group} -a "/var/lib/lc-wrapper/.venv/bin/memeid-extractor -C --strict-validate -o /dev/null"

エラーが検出されると、以下のようなメッセージが表示されます。

```
Validation errors found:
  python-basic-2025/lesson01.ipynb:
    Duplicate values: lc_cell_meme, textbook_path
    Path conflicts: python-basic-2024/lesson01.ipynb

To fix these errors, run:
  jupyter nblineage new-root-meme python-basic-2025/lesson01.ipynb python-basic-2025/lesson01.fixed.ipynb

Validation failed in strict mode. CSV generation aborted.non-zero return code
```

上記のメッセージでは`python-basic-2024/lesson01.ipynb`と`python-basic-2025/lesson01.ipynb`におけるMEME IDが重複していることを示しています。過去の教材を元にして作成したnotebookではこのようなことが起こり得ます。その場合は教材を作成した環境で以下の操作を行い、その後に教材のリリースを行なってください。

```console
jupyter nblineage new-root-meme python-basic-2025/lesson01.ipynb python-basic-2025/lesson01.fixed.ipynb
mv python-basic-2025/lesson01.fixed.ipynb python-basic-2025/lesson01.ipynb
```

MEME IDの収集はCoursewareHubの実行環境でsystemdサービスとして実行されています。サービスのログを確認します。

In [ ]:
!ansible {cwh_group} -b -a "journalctl -u update-meme-csv"

収集されたMEME IDを記録したCSVファイルを表示します。

In [ ]:
!env ANSIBLE_NOCOLOR=1 ansible {cwh_group} -a 'cat {csv_file if "csv_file" in vars() else "/var/lib/lc-wrapper/meme.csv"}'

## Elasticsearch, Kibanaの設定

### パラメータの指定

ログの保存先となるElasticsearchのインデックスのプレフィックスを次のセルで指定してください。実際の保存先となるインデックス名は、ここで指定した値の末尾に日時をつけた値となります。例えば`index_prefix`に`lcwrapper`を指定した場合は`lcwrapper-2024.04.01`のような値となります。

In [ ]:
# (例)
# index_prefix = "lcwrapper"

index_prefix = "lcwrapper"

上のセルで指定したインデックスをKibanaの[データビュー](https://www.elastic.co/guide/en/kibana/current/data-views.html)として登録します。データビューの名前を次のセルで指定してください。


In [ ]:
# (例)
# kibana_data_view = "CoursewareHub LC_wrapper log"

kibana_data_view = "CoursewareHub LC_wrapper log"

指定したパラメータをファイルに保存します。

In [ ]:
%run scripts/group.py
update_group_vars(
    cwh_group,
    _file='10-es.yml',
    index_prefix=index_prefix,
    kibana_data_view=kibana_data_view,
)

!cat group_vars/{cwh_group}/10-es.yml

### Elasticsearch

ログの保存先となるインデックスに対してテンプレートを登録します。

lc_wrapperの出力したログにはセルの実行したときに開始時刻`start_time`と終了時刻`end_time`が含まれています。これらの値をElasticsearchで日時として扱われるようにするためのテンプレートを登録します。

In [ ]:
import json

idx_template_name = 'lc-wrapper-template'
idx_template = {
    "index_patterns": [
        f"{index_prefix}*",
    ],
    "priority" : 100,
    "template": {
        "mappings": {
            "properties": {
                "start_time": {
                    "format": "epoch_second",
                    "type": "date",
                },
                "end_time": {
                    "format": "epoch_second",
                    "type": "date",
                },
            }
        }
    },
}

!curl -X PUT {es_url}/_index_template/{idx_template_name} {es_opts} \
    -d '{json.dumps(idx_template)}' \
    -H 'Content-Type: application/json'

### Kibana

ログの送信先となるインデックスとKibanaのデータビューに登録します。

In [ ]:
kibana_data = {
    "data_view": {
        "name": kibana_data_view,
        "title": f"{index_prefix}*",
        "timeFieldName": "@timestamp",
    },
}
!curl -X POST {kibana_url}/api/data_views/data_view -d '{json.dumps(kibana_data)}' {es_opts} \
    -H 'Content-Type: application/json; Elastic-Api-Version=2023-10-31' -H "kbn-xsrf: string"

登録されたことを確認します。

In [ ]:
!curl -s {es_opts} {kibana_url}/api/data_views | jq

## fluentdの設定

lc_wrapperのログを処理してElasticsearchに送信する設定をfluentdに対して行います。

### タグの指定

lc_wrapperのログに付けるfluentdのタグを指定します。

In [ ]:
# (例)
# tag = "cwh.lc_wrapper"

tag = "cwh.lc_wrapper"

指定された値を保存します。

In [ ]:
%run scripts/group.py
update_group_vars(cwh_group, _file='10-fluentd.yml', tag=tag)
!cat group_vars/{cwh_group}/10-fluentd.yml

### セル実行結果からテキスト部分の抽出

セルの実行結果はJSON形式で`result_json`フィールドに記録されます。この値のテキスト部分を抽出して`result_text`フィールドに記録します。ここでは`result_json`のどの部分を抽出対象とするかの指定を行います。

抽出対象の指定は[JSONPath](https://www.rfc-editor.org/rfc/rfc9535.html)で行います。例えば`result_json`が以下のような値であった場合を考えます。

```json
{
  "content": {
    "execution_count": 7,
    "metadata": {},
    "data": {
      "text/plain": "0.8414709848078965"
    }
  },
  "msg_type": "execute_result"
}
```

JSONPathとして`$.content.data["text/plain"]`することで`result_text`に`0.8414709848078965`が抽出されます。

次のセルで`result_text`を抽出するためのJSONPathを指定してください。複数のJSONPathを指定した場合は最初に値が得られたものを`result_text`に設定します。

In [ ]:
result2text_jsonpath = [
    '$.content.data["text/plain"]',
    '$.content.ename',
]

指定された値を保存します。

In [ ]:
%run scripts/group.py
update_group_vars(cwh_group, _file='10-fluentd.yml', result2text_jsonpath=result2text_jsonpath)
!cat group_vars/{cwh_group}/10-fluentd.yml

### 設定の更新

指定されたパラメータでfluentdとCoursewareHubの設定を更新します。

次のセルでansibleのplaybookが実行するとCoursewareHubとfluentdの設定が更新されます。具体的には以下の操作が行われます。

* CoursewareHubの設定
    * 設定ファイル`/etc/jupyterhub/jupyterhub_config.d/lc_wrapper.py`を配置する
        * 環境変数を設定しsingle-userサーバのlc_wrapperからfluentdにログを送信するようにする
    * 設定を反映するためにJupyterHubコンテナを再起動する
* fluentdの設定
    * fluentdプラグインのgemをインストールする
    * 設定ファイル`/etc/fluentd/config.d/lc_wrapper.conf`を配置する
    * fluentdに設定ファイルの再読み込みを指示する

In [ ]:
!ansible-playbook -l {cwh_group} playbooks/deploy-lc_wrapper.yml

## Kibanaによる可視化

ここまでの設定で、single-userサーバのlc_wrapperログがElasticsearchで収集されるようになります。また、ログの送信先となるインデックスパターンがKibanaのデータビューとして登録されています。

Kibanaのメニューから[Management/Stack Management]--[Kibana/Data View]を選択して、データビューが登録されていることを確認してください。

登録されたデータビューをKibanaのDiscoverで表示すると次図のようになります。

![Kibana-Discover](images/fluentd-201-11.png)

### 可視化に利用できる情報

ログから各ユーザのnotebook実行状況を可視化するにはユーザとコードセルに関する情報が必要になります。それぞれの情報に関するフィールドとしてどのようなものがあるのかについて以下にまとめました。

Elasticsearchで収集したログの詳細については[1.3 ログデータ](#ログデータ)を参照してください。

#### ユーザに関するフィールド名

|フィールド名|説明|例|備考|
|:---|:---|:---|:---|
|user_name|アカウント名|user01xacca0c|
|local_name|メールアドレスのローカルパート|user01|複数のユーザで重複する可能性がある|
|uid|利用者のUID|1001|

#### コードセルに関するフィールド名

|フィールド名|説明|例|備考|
|:---|:---|:---|:---|
|qno|コードセルの通し番号|00-000|リリースした教材が変更されると、同じセルに異なる番号が割り当てらる可能性がある|
|qlabel|実行したセルの直前にあるmarkdownセルから抽出した文字列|問1-1|
|lc_cell_meme|セルのMEME ID|1a339f04-3e8f-11ef-91c2-02420a640009-3-c957-e4b7-54b0|
|lc_cell_meme_uuid|セルのMEME IDからブランチ番号を除いた値|1a339f04-3e8f-11ef-91c2-02420a640009|

### ダッシュボードの登録

各ユーザのnotebook実行状況を可視化するためのダッシュボードをKibanaに登録します。

Kibanaのメニューから[Dashboards]--[Create Dashboard]--[Create Visualization]を選択してください。下図のような画面が表示されるのでパネルを作成します。

![Create Visualization](images/fluentd-201-12.png)

ユーザのnotebook実行状況を可視化する例としてヒートマップを用いる場合と、積み上げ棒グラフを用いる場合について、以下で説明します。

#### ヒートマップ

それぞれのユーザがどのコードセルを実行したのかをヒートマップで可視化します。

縦軸、横軸にユーザ名、コードセルの通番とするヒートマップを作成します。視覚化パネルの設定で以下に示すようなパラメータを指定します。

* Visualization type
    * Heat map
* Horizontal axis
    * Functions: Top values
    * Fields: qno.keyword
    * Number of values: 100
    * Rank by: Alphabetical
* Vertical axis
    * Functions: Top values
    * Fields: local_name.keyword
    * Number of values: 100
    * Rank by: Alphabetical
* Cell value
    * Functions: count
    * Field: Records

またKibanaの[time range設定](https://www.elastic.co/guide/en/kibana/current/set-time-filter.html)でRefresh everyを設定してください。

![time range](images/fluentd-201-13.png)

作成したヒートマップは次図のようになります。ヒートマップの色はsingle-userサーバでコードセルを実行した回数で変わります。
    
![heatmap](images/fluentd-201-14.gif)
  

可視化パネルにあるCell valueのAdvanced設定で可視化対象を絞り込むことができます。例えば、セルの実行結果が成功したものだけを対象にする場合はFilter by欄に`execute_reply_status.keyword:"ok"`を指定します。

![Advanced](images/fluentd-201-15.png)

#### 積み上げ棒グラフ

それぞれのユーザが実行したコードセルの数を積み上げ棒グラフで可視化します。

縦軸、横軸にユーザ名、コードセルの通番とするヒートマップを作成します。視覚化パネルの設定で以下に示すようなパラメータを指定します。

* Visualization type
    * Bar horizontal stacked
* Vertical axis
    * Functions: Top values
    * Fields: local_name.keyword
    * Number of values: 100
    * Rank by: Alphabetical
* Horizontal axis
    * Functions: Unique Count
    * Field: qno.keyword
* Breakdown
    * Functions: Top values
    * Fields: qno.keyword
    * Functions: Top values
    * Number of values: 100
    * Rank by: Alphabetical

またKibanaの[time range設定](https://www.elastic.co/guide/en/kibana/current/set-time-filter.html)でRefresh everyを設定してください。

作成した積み上げ棒グラフは次図のようになります。
    
![bar horizontal stacked](images/fluentd-201-16.gif)
  